## Clean the raw data

Load Packages

In [21]:
import pandas as pd
import numpy as np
from datetime import datetime

Read File

In [22]:
path = 'raw.csv'
Russell_data = pd.read_csv(path)

Get rid of row which contains average and delete the "Versus" column

In [23]:
Russell_data = Russell_data[Russell_data['Date'] != "Average"]

Russell_data = Russell_data.drop("Versus", axis=1)

I assueme the 3P% and FT% to be zero if they didn't make any 3PA or FTA

In [24]:
Russell_data['3P%'].fillna(0, inplace=True)
Russell_data['FT%'].fillna(0, inplace=True)

Drop invalid value

In [25]:
Russell_data = Russell_data.dropna()   # Actually, there is no na value.

Split the score to three parts (Win/Lose; Team score; Opponent score)

In [26]:
Russell_data['Win/Lose'] = Russell_data['Score'].str[0]

score_split = Russell_data['Score'].str[1:].str.split('-', expand=True)

Russell_data['Team score'] = score_split[0]

Russell_data['Opponent score'] = score_split[1]

# Drop the original column of score
Russell_data = Russell_data.drop("Score", axis=1)


Split Date

In [27]:
Russell_data['DayOfWeek'] = Russell_data['Date'].str[:3]  # First three letters
Russell_data['date'] = Russell_data['Date'].str[3:]  # Rest of the string
Russell_data = Russell_data.drop("Date", axis=1)

Check data type and make score to be int and removing trailing spaces

In [28]:
Russell_data['Team score'] = Russell_data['Team score'].astype('Int64')
Russell_data['Opponent score'] = Russell_data['Opponent score'].astype('Int64')

for column in Russell_data.select_dtypes(include=['object']):  # Select only columns with object dtype
    Russell_data[column] = Russell_data[column].str.strip()
    

Combine "year" with the "date"

In [29]:
Russell_data['Year'] = Russell_data['Year'].astype(int).astype(str)



Russell_data['date'] = Russell_data['date'] + '/' + Russell_data['Year']
#Russell_data['date'] = Russell_data['date'].str.rstrip('.0')

Russell_data['date'] = pd.to_datetime(Russell_data['date'], format='%m/%d/%Y', errors='coerce')


In [30]:
invalid_dates = Russell_data[Russell_data['date'].isna()]
invalid_dates[['date', 'Year']]

,date,Year


Add column "Season"

In [31]:
# Calculate the 'Season' based on the year
Russell_data['Season'] = Russell_data['Year'].astype(int) - 2008  # Subtracting the base year from each entry

Make sure the date is accurate 

In [32]:
Russell_data['date'] = np.where(
    Russell_data['date'].dt.month > 9,
    Russell_data['date'] - pd.offsets.DateOffset(years=1),
    Russell_data['date']
)
Russell_data['date']

Russell_data['Year'] = Russell_data['date'].dt.year

Sort data by date

In [33]:
Russell_data.sort_values(by="date", inplace=True)

In [34]:
unique_years = Russell_data['Year'].unique()
year_counts = Russell_data['Year'].value_counts(sort=False)

print("Unique years in the dataset:", unique_years)
print("Counts for each year:")
print(year_counts)


Unique years in the dataset: [2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021
 2022 2023 2024]
Counts for each year:
2008     33
2009     81
2010     90
2011     70
2012    111
2013     79
2014     59
2015     81
2016     99
2017     89
2018     77
2019     80
2020     39
2021    105
2022     74
2023     76
2024     11
Name: Year, dtype: int64


Classify the playoff game and regular season game

In [35]:
# Set the default value for all rows in the new column
Russell_data["Game Type"] = "Regular"

# Initialize a counter for consecutive occurrences
consecutive_count = 1

# Store the index of the first game in the current sequence of games against the same opponent
start_index = None

previous_opponent = None

# Track the date of last playoff game
# last_playoff_date = None

# Use iterrows to safely iterate over DataFrame rows
for i, row in Russell_data.iterrows():
    if start_index is None:
        start_index = i  # Initialize start_index with the first row's index

    if row['Opponent'] == previous_opponent:
        consecutive_count += 1
        if consecutive_count >= 4:
                # if last_playoff_date is not None:
                # month_diff = (row['Date'].year - last_playoff_date.year) * 12 + row['Date'].month - last_playoff_date.month
                # if month_diff > 2:
                #     # If more than 2 months apart, it's a regular game
                #     Russell_data.at[i, 'Game Type'] = 'Regular'
                #     # Reset the last playoff date
                #     last_playoff_date = None
                #     continue
            # When we have at least 4 consecutive games, mark them as 'Playoff'
            Russell_data.loc[start_index:i, 'Game Type'] = 'Playoff'
    else:
        # Reset the counter and start_index when a new opponent is encountered
        consecutive_count = 1
        start_index = i
        previous_opponent = row['Opponent']
        
        
#2020 3 games are playoff that doesn't count 
#2013 2 games are playoff that doesn't count 
dates_to_change = ['2020-08-29', '2020-08-31', '2020-09-02',"2013-04-21","2013-04-24"]
dates_to_change = pd.to_datetime(dates_to_change)

for date in dates_to_change:
    Russell_data.loc[Russell_data['date'] == date, 'Game Type'] = 'Playoff'


In [36]:
# Assuming 'Russell_data' is your dataframe and is already loaded with data similar to the provided image.

# Group the data by 'Year' and filter for 'Playoff' games, then count the number of such games for each year.
playoff_counts_by_year = Russell_data[Russell_data["Game Type"] == "Playoff"].groupby('Year').size()

# Print the results
for year, count in playoff_counts_by_year.items():
    print(f"{year}: {count} playoff games")

2010: 6 playoff games
2011: 17 playoff games
2012: 20 playoff games
2013: 2 playoff games
2014: 19 playoff games
2016: 18 playoff games
2017: 5 playoff games
2018: 6 playoff games
2019: 5 playoff games
2020: 8 playoff games
2021: 5 playoff games
2023: 6 playoff games


Change the "Win/Lose" column type.(Change string to int)  0:L  1:W

In [37]:
Russell_data['Win/Lose'] = Russell_data['Win/Lose'].map({'L': 0, 'W': 1})

Change the "DayOfWeek" type. eg. Mon - 1; Tue - 2, etc

In [38]:
Russell_data['DayOfWeek'] = Russell_data['DayOfWeek'].map({'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6, 'Sun': 7})
Russell_data

,Team,Opponent,Min,FGM,FGA,FG%,3PM,3PA,3P%,FTM,...,PTS,+/-,Year,Win/Lose,Team score,Opponent score,DayOfWeek,date,Season,Game Type
87,OKC,MIL,22.0,4.0,9.0,44.4,1.0,2.0,50.0,4.0,...,13.0,5.0,2008,0,87,98,3,2008-10-29,1,Regular
85,OKC,HOU,19.0,2.0,7.0,28.6,0.0,1.0,0.0,2.0,...,6.0,-19.0,2008,0,77,89,6,2008-11-01,1,Regular
84,OKC,MIN,25.0,6.0,13.0,46.2,0.0,1.0,0.0,2.0,...,14.0,7.0,2008,1,88,85,7,2008-11-02,1,Regular
83,OKC,BOS,19.0,4.0,13.0,30.8,3.0,5.0,60.0,2.0,...,13.0,-3.0,2008,0,83,96,3,2008-11-05,1,Regular
82,OKC,UTA,17.0,1.0,8.0,12.5,0.0,0.0,0.0,4.0,...,6.0,-5.0,2008,0,97,104,5,2008-11-07,1,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,LAC,MEM,16.0,5.0,11.0,45.5,1.0,3.0,33.3,1.0,...,12.0,0.0,2024,1,128,119,5,2024-01-12,16,Regular
1338,LAC,MIN,29.0,3.0,8.0,37.5,2.0,2.0,100.0,4.0,...,12.0,2.0,2024,0,105,109,7,2024-01-14,16,Regular
1337,LAC,OKC,22.0,4.0,8.0,50.0,0.0,3.0,0.0,3.0,...,11.0,6.0,2024,1,128,117,2,2024-01-16,16,Regular
1336,LAC,BKN,31.0,10.0,16.0,62.5,1.0,2.0,50.0,2.0,...,23.0,22.0,2024,1,125,114,7,2024-01-21,16,Regular


Create new CSV file


In [39]:
Russell_data.to_csv('Reorgnized_RB.csv', index=False)